In [1]:
import warnings
warnings.filterwarnings('ignore')

_EPSILON = 1e-08

import numpy as np
import pandas as pd
import tensorflow as tf
import random
import os

from tensorflow.contrib.layers import fully_connected as FC_Net
from sklearn.metrics import brier_score_loss
from sklearn.model_selection import train_test_split

from sksurv.metrics import concordance_index_ipcw, concordance_index_censored, brier_score

from helpers.data_loader import *
from helpers.utils_others import *

from models.model import *
from helpers.utils_others import *

### IMPORT DATASET

In [2]:
OUT_ITERATION               = 5
MICE_IMPUTED_DATA_VERSION   = False
data_mode                   = 'STRATCANS_v1_new2'\
    + ('_MICE' if MICE_IMPUTED_DATA_VERSION else '')
seed                        = 1234

(data_xs, data_xt, data_time, data_y, data_tte), \
(feat_static, feat_timevarying), \
(xt_bin_list, xt_con_list) = \
import_dataset_STRATCANS_v1(mice_version=MICE_IMPUTED_DATA_VERSION)


# removes first and secondary pearson score
data_xs           = data_xs[:, [0,1,2,3,4,5,6]]
feat_static       = feat_static[[0,1,2,3,4,5,6]]

data_xt           = data_xt[:, :, [0,1,2,3,6,7,8,9,10]]
feat_timevarying  = feat_timevarying[[0,1,2,3,6,7,8,9,10]]
xt_con_list       = [0, 1, 2, 3, 4, 5, 6, 7]

x_dim_static      = len(feat_static)
x_dim_timevarying = len(feat_timevarying) # this includes delta

max_length                  = np.shape(data_time)[1]
num_Event                   = len(np.unique(data_y)) - 1  #number of next outcome events

data_y_new   = np.zeros([np.shape(data_y)[0], max_length, num_Event])
data_tte_new = np.zeros([np.shape(data_y)[0], max_length, num_Event])

seq_length = np.sum(np.sum(data_xt, axis=2) != 0, axis=1)

for i in range(np.shape(data_y)[0]):
    data_y_new[i, :seq_length[i], :]   = data_y[i]
    data_tte_new[i, :seq_length[i], :] = data_tte[i]

In [3]:
#====================================================
##### HYPER-PARAMETERS
mb_size                     = 64

iteration                   = 50000

keep_prob                   = 0.6
lr_train                    = 1e-3

h_dim_RNN                   = 100
h_dim_FC                    = 100

num_layers_RNN              = 2
num_layers_FC               = 3


RNN_type                    = 'GRU' #GRU, LSTM
BiRNN                       = None #if not 'None''BiRNN'

FC_active_fn                = tf.nn.relu

initial_W                   = tf.contrib.layers.xavier_initializer()

alpha                       = 10.0  #for stepahead-prediction loss
p_weibull                   = 1.0 #1.1  #1.0 # 0.5 or 1.0 performed good...


reg_scale                   = 0.


##### MAKE DICTIONARIES
# INPUT DIMENSIONS
input_dims                  = { 'x_dim_static'      : x_dim_static,
                                'x_dim_timevarying' : x_dim_timevarying, #this includes delta
                                'num_Event'         : num_Event,         #next-event types
                                'xt_con_list'       : xt_con_list,
                                'xt_bin_list'       : xt_bin_list,
                                'max_length'        : max_length }

# NETWORK HYPER-PARMETERS
network_settings            = { 'p_weibull'         : p_weibull,
                                'h_dim_RNN'         : h_dim_RNN,
                                'h_dim_FC'          : h_dim_FC,
                                'num_layers_RNN'    : num_layers_RNN,
                                'num_layers_FC'     : num_layers_FC,
                                'RNN_type'          : RNN_type,
                                'BiRNN'             : BiRNN,
                                'FC_active_fn'      : FC_active_fn,
                                'RNN_active_fn'     : tf.nn.tanh,
                                'initial_W'         : initial_W,
                                'reg_scale'         : reg_scale}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
seed = 1234
out_itr = 0

In [5]:
EVAL_TIMES = [180, 365*1, 365*2, 365*3]

FINAL_C = np.zeros([OUT_ITERATION, len(EVAL_TIMES), num_Event])
FINAL_B = np.zeros([OUT_ITERATION, len(EVAL_TIMES), num_Event])

In [6]:
PRED_TIMES = [0, 365, 365*2, 365*3]
EVAL_TIMES = [365, 365*2, 365*3, 365*4, 365*5, 365*6, 365*7, 365*8, 365*9, 365*10]

FINAL_RESULT1 = np.zeros([OUT_ITERATION, len(PRED_TIMES), len(EVAL_TIMES)])
FINAL_RESULT2 = np.zeros([OUT_ITERATION, len(PRED_TIMES), len(EVAL_TIMES)])

FINAL_RESULT1_pred = np.zeros([OUT_ITERATION, len(PRED_TIMES), len(EVAL_TIMES)])
FINAL_RESULT2_pred = np.zeros([OUT_ITERATION, len(PRED_TIMES), len(EVAL_TIMES)])

# for out_itr in range(1):
#     out_itr = 0
for out_itr in range(OUT_ITERATION):  
    save_path = './{}/p{}/itr{}'.format(data_mode, p_weibull, out_itr)

    if not os.path.exists(save_path + '/models/'):
        os.makedirs(save_path + '/models/')

    if not os.path.exists(save_path + '/results/'):
        os.makedirs(save_path + '/results/')
    
    (tr_data_s,te_data_s, tr_data_t,te_data_t, tr_time,te_time, tr_tte,te_tte, tr_label,te_label, tr_tte_new,te_tte_new, tr_label_new,te_label_new) = train_test_split(
        data_xs, data_xt, data_time, data_tte, data_y, data_tte_new, data_y_new, test_size=0.2, random_state=seed+out_itr
    ) 

    (tr_data_s,va_data_s, tr_data_t,va_data_t, tr_time,va_time, tr_tte,va_tte, tr_label,va_label, tr_tte_new,va_tte_new, tr_label_new,va_label_new) = train_test_split(
        tr_data_s, tr_data_t, tr_time, tr_tte, tr_label, tr_tte_new, tr_label_new, test_size=0.2, random_state=seed+out_itr
    )
    
    tf.reset_default_graph()
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.45)
    config      = tf.ConfigProto(gpu_options=gpu_options)
#     config.gpu_options.allow_growth = True
    sess        = tf.Session(config=config)
    model       = Model_DeepHit_Weibull(sess, "Version1", input_dims, network_settings)

    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())

    CHECK_STEP = 100

    min_loss = 1e8
    max_flag = 50

    avg_loss_tte   = 0   
    avg_loss_mle   = 0
    stopflag = 0

    print( "MAIN TRAINING ...")
    for itr in range(iteration):
        xs_mb, xt_mb, t_mb, tte_mb, m_mb = f_get_minibatch(
            mb_size, tr_data_s, tr_data_t, tr_time, tr_tte_new, tr_label_new
        )

        DATA       = (xs_mb, xt_mb, t_mb, tte_mb, m_mb)

        _, loss_tte, loss_mle = model.train(DATA, alpha, keep_prob, lr_train)
        
        avg_loss_tte   += loss_tte/CHECK_STEP
        avg_loss_mle   += loss_mle/CHECK_STEP

        if (itr+1)%CHECK_STEP == 0:
            stopflag += 1

            DATA    = (va_data_s, va_data_t, va_time, va_tte_new, va_label_new)

            va_loss_tte, va_loss_mle = model.get_cost(DATA, alpha)

            print('|| Epoch:{0:05d} | TR_Loss_tte:{1:0.4f} |  TR_Loss_mle:{2:0.4f} || VA_Loss_tte:{3:0.4f} || VA_Loss_mle:{4:0.4f} ||'.format(
                itr + 1, avg_loss_tte, avg_loss_mle, va_loss_tte, va_loss_mle))

            avg_loss_tte  = 0   
            avg_loss_mle  = 0
            
            if min_loss > va_loss_tte:
                stopflag = 0
                min_loss = va_loss_tte
                
#             if min_loss > va_loss_mle:
#                 stopflag = 0
#                 min_loss = va_loss_mle

                saver.save(sess, save_path + '/models/model_tte')
                print('saved...')

#                 tmp_pred, _ = f_get_prediction(model, va_data_s, va_data_t, va_time, EVAL_TIMES)
#                 print('saved... | max prediction: {}'.format(np.max(np.max(tmp_pred, axis=2), axis=0)) )     
    


                RESULT1 = np.zeros([len(PRED_TIMES), len(EVAL_TIMES)])
                RESULT2 = np.zeros([len(PRED_TIMES), len(EVAL_TIMES)])

                for p_idx, pred_time in enumerate(PRED_TIMES):
                    p_idx_te  = te_tte >= pred_time
                    p_idx_tr  = tr_tte >= pred_time

                    tmp_pred, tmp_idx = f_get_prediction(model, te_data_s, te_data_t, te_time, EVAL_TIMES, pred_time_= pred_time)


                    tmp_y = tr_label[p_idx_tr]
                    tmp_t = tr_tte[p_idx_tr] - pred_time

                    tr_y_structured =  [(tmp_y[i], tmp_t[i]) for i in range(len(tmp_y))]
                    tr_y_structured = np.array(tr_y_structured, dtype=[('status', 'bool'),('time','<f8')])

                    tmp_y = te_label[p_idx_te]
                    tmp_t = te_tte[p_idx_te] - pred_time

                    te_y_structured =  [(tmp_y[i], tmp_t[i]) for i in range(len(tmp_y))]
                    te_y_structured = np.array(te_y_structured, dtype=[('status', 'bool'),('time','<f8')])


                    for e_idx, eval_time in enumerate(EVAL_TIMES):
                        if np.sum((tmp_t<=eval_time) & (tmp_y==1)) > 0:
                            RESULT1[p_idx, e_idx] = concordance_index_ipcw(tr_y_structured, te_y_structured, tmp_pred[p_idx_te][:, e_idx], tau=eval_time)[0]
                            RESULT2[p_idx, e_idx] = brier_score(tr_y_structured, te_y_structured, 1.- tmp_pred[p_idx_te][:, e_idx], times=eval_time)[1][0]
                        
                print(RESULT1)
                
        if stopflag >= max_flag:
            print('model trained...')
            break
            
    saver.restore(sess, save_path + '/models/model_tte')

    PRED_TIMES = [0, 365, 365*2, 365*3]
    EVAL_TIMES = [365, 365*2, 365*3, 365*4, 365*5, 365*6, 365*7, 365*8, 365*9, 365*10]

    RESULT1 = -1. * np.ones([len(PRED_TIMES), len(EVAL_TIMES)])
    RESULT2 = -1. * np.ones([len(PRED_TIMES), len(EVAL_TIMES)])

    for p_idx, pred_time in enumerate(PRED_TIMES):
        p_idx_te  = te_tte >= pred_time
        p_idx_tr  = tr_tte >= pred_time

        tmp_pred, tmp_idx = f_get_prediction(model, te_data_s, te_data_t, te_time, EVAL_TIMES, pred_time_= pred_time)


        tmp_y = tr_label[p_idx_tr]
        tmp_t = tr_tte[p_idx_tr] - pred_time

        tr_y_structured =  [(tmp_y[i], tmp_t[i]) for i in range(len(tmp_y))]
        tr_y_structured = np.array(tr_y_structured, dtype=[('status', 'bool'),('time','<f8')])

        tmp_y = te_label[p_idx_te]
        tmp_t = te_tte[p_idx_te] - pred_time

        te_y_structured =  [(tmp_y[i], tmp_t[i]) for i in range(len(tmp_y))]
        te_y_structured = np.array(te_y_structured, dtype=[('status', 'bool'),('time','<f8')])


        for e_idx, eval_time in enumerate(EVAL_TIMES):
            if np.sum((tmp_t<=eval_time) & (tmp_y==1)) > 0:
                RESULT1[p_idx, e_idx] = concordance_index_ipcw(tr_y_structured, te_y_structured, tmp_pred[p_idx_te][:, e_idx], tau=eval_time)[0]
                RESULT2[p_idx, e_idx] = brier_score(tr_y_structured, te_y_structured, 1.- tmp_pred[p_idx_te][:, e_idx], times=eval_time)[1][0]


    pd.DataFrame(RESULT1, index=PRED_TIMES, columns=EVAL_TIMES).to_csv(save_path+'/results/c_index.csv')
    pd.DataFrame(RESULT2, index=PRED_TIMES, columns=EVAL_TIMES).to_csv(save_path+'/results/brier_score.csv')
    
    FINAL_RESULT1[out_itr, :, :] = RESULT1
    FINAL_RESULT2[out_itr, :, :] = RESULT2
    
    
    RESULT1 = -1. * np.ones([len(PRED_TIMES), len(EVAL_TIMES)])
    RESULT2 = -1. * np.ones([len(PRED_TIMES), len(EVAL_TIMES)])

    for p_idx, pred_time in enumerate(PRED_TIMES):
        p_idx_te  = te_tte >= 0
        p_idx_tr  = tr_tte >= 0

        tmp_pred, tmp_idx = f_get_prediction(model, te_data_s, te_data_t, te_time, EVAL_TIMES, pred_time_= pred_time)


        tmp_y = tr_label[p_idx_tr]
        tmp_t = tr_tte[p_idx_tr] - 0

        tr_y_structured =  [(tmp_y[i], tmp_t[i]) for i in range(len(tmp_y))]
        tr_y_structured = np.array(tr_y_structured, dtype=[('status', 'bool'),('time','<f8')])

        tmp_y = te_label[p_idx_te]
        tmp_t = te_tte[p_idx_te] - 0

        te_y_structured =  [(tmp_y[i], tmp_t[i]) for i in range(len(tmp_y))]
        te_y_structured = np.array(te_y_structured, dtype=[('status', 'bool'),('time','<f8')])


        for e_idx, eval_time in enumerate(EVAL_TIMES):
            if np.sum((tmp_t<=eval_time) & (tmp_y==1)) > 0:
                RESULT1[p_idx, e_idx] = concordance_index_ipcw(tr_y_structured, te_y_structured, tmp_pred[p_idx_te][:, e_idx], tau=eval_time)[0]
                RESULT2[p_idx, e_idx] = brier_score(tr_y_structured, te_y_structured, 1.-tmp_pred[p_idx_te][:, e_idx], times=eval_time)[1][0]

    pd.DataFrame(RESULT1, index=PRED_TIMES, columns=EVAL_TIMES).to_csv(save_path+'/results/c_index_pred.csv')
    pd.DataFrame(RESULT2, index=PRED_TIMES, columns=EVAL_TIMES).to_csv(save_path+'/results/brier_score_pred.csv')
    
    FINAL_RESULT1_pred[out_itr, :, :] = RESULT1
    FINAL_RESULT2_pred[out_itr, :, :] = RESULT2
    
    save_path2 = './{}/p{}/'.format(data_mode, p_weibull)

FINAL_RESULT1[FINAL_RESULT1 == -1] = np.nan
FINAL_RESULT2[FINAL_RESULT2 == -1] = np.nan

pd.DataFrame(np.nanmean(FINAL_RESULT1, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv(save_path2 + 'final_c_index_mean.csv')
pd.DataFrame(np.nanmean(FINAL_RESULT2, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv(save_path2 + 'final_brier_score_mean.csv')

pd.DataFrame(np.nanstd(FINAL_RESULT1, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv(save_path2 + 'final_c_index_std.csv')
pd.DataFrame(np.nanstd(FINAL_RESULT2, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv(save_path2 + 'final_brier_score_std.csv')


FINAL_RESULT1_pred[FINAL_RESULT1_pred == -1] = np.nan
FINAL_RESULT2_pred[FINAL_RESULT2_pred == -1] = np.nan

pd.DataFrame(np.nanmean(FINAL_RESULT1_pred, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv(save_path2 + 'final_c_index_pred_mean.csv')
pd.DataFrame(np.nanmean(FINAL_RESULT2_pred, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv(save_path2 + 'final_brier_score_pred_mean.csv')

pd.DataFrame(np.nanstd(FINAL_RESULT1_pred, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv(save_path2 + 'final_c_index_pred_std.csv')
pd.DataFrame(np.nanstd(FINAL_RESULT2_pred, axis=0), index=PRED_TIMES, columns=EVAL_TIMES).to_csv(save_path2 + 'final_brier_score_pred_std.csv')

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.



Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Deprecated in fav